# Naive Chunking

Apply chunking and THEN compute embeddings.

Here the result is incorrect. You can compare it to late chunking to see the difference.

In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from nlp_chat_bot.model.embedding.minilm import MiniLM
from nlp_chat_bot.rag.classic_rag import ClassicRAG
from nlp_chat_bot.vector_store.naive_chunking_chroma_vector_store_builder import NaiveChunkingChromaVectorStoreBuilder

d:\Programs\Anaconda\envs\nlp_project_chatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
dataset_path = "../data"
vector_store_path = "../chromadb"
model_download_path = "../models"

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=50,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)

embedding_function = MiniLM(model_download_path=model_download_path)
llm_gemini = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
document_loader = None
vector_store = NaiveChunkingChromaVectorStoreBuilder(dataset_path,
                                        embedding_function,
                                        vector_store_path,
                                        splitter).build()
rag = ClassicRAG(vector_store, llm_gemini)

print("LENGTH", rag.get_num_docs())
docs_retrieved = rag.retrieve(state = {"question": "What is my conclusion in my project report?", "context": []})

print("Num docs:", len(docs_retrieved["context"]))


for i in range(len(docs_retrieved["context"])):
    doc = docs_retrieved["context"][i]
    print("\n\n", "#"*30,"\n")
    print(f"doc {i}: (score: {doc.metadata['score']})")
    print(doc.page_content)

100%|██████████| 1/1 [00:00<00:00,  6.36it/s]
0it [00:00, ?it/s]
100%|██████████| 1/1 [00:02<00:00,  2.02s/it]
0it [00:00, ?it/s]
Storing documents embeddings (batch size is 1000): 0it [00:00, ?it/s]
d:\Programs\Anaconda\envs\nlp_project_chatbot\lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Documents are now loaded
LENGTH 35
Num docs: 3


 ############################## 

doc 0: (score: 1.6559454535018854)
Privacy policy About Wikipedia Disclaimers Contact Wikipedia Code of Conduct Developers Statistics Cookie statement Mobile view

This page was last edited on 12 December 2024, at 00:08 (UTC).

Text is available under the Creative Commons Attribution-ShareAlike 4.0 License; additional terms may apply. By using this site, you agree to the Terms of Use and Privacy Policy. Wikipedia® is a registered trademark of the Wikimedia Foundation, Inc., a non-profit organization.

Contents

hide

(Top)

Background

Capabilities

Corporate customization

GPT-4o mini

Scarlett Johansson controversy

See also

References

GPT-4o

17 languages

Article Talk Read Edit View history

Tools

Appearance

hide

Text

Width

Color (beta)

Small

Standard

Large

Standard

Wide

Automatic

Light

Dark

Generative Pre-trained Transformer

4 Omni (GPT-4o)

Developer(s) OpenAI

Initial release May 

In [5]:
rag.invoke(query={"question":"What is my conclusion in my project report?"})["answer"]

'I am sorry, but this document does not contain your project report, therefore I cannot answer your question.  The provided text is about GPT-4o and related articles.\n'